In [1]:
import os

In [2]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Bulldozer-Prediction\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\ainao\\Downloads\\Projects\\Bulldozer-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path
import yaml

@dataclass(frozen=True)
class DataIngestionConfig:
    train_path : Path
    source_train_path: Path
    source_test_path: Path
    test_path : Path
    root_dir: Path
    

In [6]:
import sys
import os

sys.path.append(os.path.abspath("src"))
from bullprediction.utils.common import read_yaml, create_directories
from bullprediction.constants import *

In [7]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH):

        self.config = read_yaml(config_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_ingestion(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
        root_dir = config.root_dir,
        source_train_path = config.source_train_path,
        source_test_path = config.source_test_path,
        train_path = config.train_path,
        test_path = config.test_path
        )

        return data_ingestion_config    
            
        
    

In [8]:
import pandas as pd

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config =config

    

    def read_data(self):
        self.df_train = pd.read_csv(self.config.source_train_path,low_memory=False,parse_dates=['saledate'])
        self.df_test = pd.read_csv(self.config.source_test_path,low_memory=False,parse_dates=['saledate'])

    def add_date_features(self, df:pd.DataFrame) -> pd.DataFrame:
        df = df.sort_values(by=['saledate'],ascending=True).copy()
        df["saleYear"] = df.saledate.dt.year
        df["saleMonth"] = df.saledate.dt.month
        df["saleDay"] = df.saledate.dt.day
        df["saleDayOfWeek"] = df.saledate.dt.dayofweek
        df["saleDayOfYear"] = df.saledate.dt.dayofyear
        df = df.drop("saledate", axis=1)
        return df
    
    def process_and_save(self):
        self.df_train = self.add_date_features(self.df_train)
        self.df_test = self.add_date_features(self.df_test)

        self.df_train.to_csv(self.config.train_path,index=False)
        self.df_test.to_csv(self.config.test_path, index=False)

    
    




In [10]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.read_data()
    data_ingestion.process_and_save()
except Exception as e:
    raise e


[2025-06-26 21:14:39,041: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-06-26 21:14:39,048: INFO: common: created directory at: artifacts]
[2025-06-26 21:14:39,051: INFO: common: created directory at: artifacts/data_ingestion]
